<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#imports" data-toc-modified-id="imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>imports</a></span></li><li><span><a href="#load-data" data-toc-modified-id="load-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>load data</a></span><ul class="toc-item"><li><span><a href="#read-in-the-files" data-toc-modified-id="read-in-the-files-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>read in the files</a></span><ul class="toc-item"><li><span><a href="#data-from-linear-and-relu-activations" data-toc-modified-id="data-from-linear-and-relu-activations-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>data from linear and relu activations</a></span></li></ul></li><li><span><a href="#create-pandas-table" data-toc-modified-id="create-pandas-table-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>create pandas table</a></span></li></ul></li><li><span><a href="#filter-out-bad-training-losses" data-toc-modified-id="filter-out-bad-training-losses-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>filter out bad training losses</a></span></li><li><span><a href="#determine-the-lambda-parameter-that-gets-the-best-test-MSE-for-each-(r,n,L)" data-toc-modified-id="determine-the-lambda-parameter-that-gets-the-best-test-MSE-for-each-(r,n,L)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>determine the lambda parameter that gets the best test MSE for each (r,n,L)</a></span></li><li><span><a href="#Generalization-MSE" data-toc-modified-id="Generalization-MSE-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Generalization MSE</a></span><ul class="toc-item"><li><span><a href="#generate-data" data-toc-modified-id="generate-data-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>generate data</a></span></li><li><span><a href="#compute-MSE" data-toc-modified-id="compute-MSE-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>compute MSE</a></span></li></ul></li><li><span><a href="#Out-of-Distribution-MSE" data-toc-modified-id="Out-of-Distribution-MSE-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Out of Distribution MSE</a></span><ul class="toc-item"><li><span><a href="#generate-data" data-toc-modified-id="generate-data-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>generate data</a></span></li><li><span><a href="#compute-MSE" data-toc-modified-id="compute-MSE-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>compute MSE</a></span></li></ul></li><li><span><a href="#Active-Subspace" data-toc-modified-id="Active-Subspace-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Active Subspace</a></span><ul class="toc-item"><li><span><a href="#evaluate-gradients-and-compute-singular-values-and-active-subspaces" data-toc-modified-id="evaluate-gradients-and-compute-singular-values-and-active-subspaces-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>evaluate gradients and compute singular values and active subspaces</a></span></li><li><span><a href="#plot-of-singular-values" data-toc-modified-id="plot-of-singular-values-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>plot of singular values</a></span></li></ul></li><li><span><a href="#determine-the-L-parameter-that-gets-the-best-test-MSE-for-each-(r,n)" data-toc-modified-id="determine-the-L-parameter-that-gets-the-best-test-MSE-for-each-(r,n)-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>determine the L parameter that gets the best test MSE for each (r,n)</a></span></li><li><span><a href="#Plots-of-L-vs-Test-error-and-n-vs-Generalization-metrics-with/without-linear-layers" data-toc-modified-id="Plots-of-L-vs-Test-error-and-n-vs-Generalization-metrics-with/without-linear-layers-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Plots of L vs Test error and n vs Generalization metrics with/without linear layers</a></span></li><li><span><a href="#Final-Table" data-toc-modified-id="Final-Table-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Final Table</a></span></li><li><span><a href="#Training-Time-Plots" data-toc-modified-id="Training-Time-Plots-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Training Time Plots</a></span><ul class="toc-item"><li><span><a href="#Train-MSE-v-Epoch" data-toc-modified-id="Train-MSE-v-Epoch-11.1"><span class="toc-item-num">11.1&nbsp;&nbsp;</span>Train MSE v Epoch</a></span></li><li><span><a href="#Weight-Decay-v-Epoch" data-toc-modified-id="Weight-Decay-v-Epoch-11.2"><span class="toc-item-num">11.2&nbsp;&nbsp;</span>Weight Decay v Epoch</a></span></li><li><span><a href="#learning-rates" data-toc-modified-id="learning-rates-11.3"><span class="toc-item-num">11.3&nbsp;&nbsp;</span>learning rates</a></span></li></ul></li></ul></div>

- 1024 up to 9 layers
    - checkpoints

# imports

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from scipy.stats import ortho_group
from scipy.stats import linregress
from scipy import linalg as la
from torch import nn
import torch

# load data

## read in the files

In [2]:
rnvals = [(1,64),(1,128),(1,256),(1,512),(1,1024),(1,2048),
          (2,64),(2,128),(2,256),(2,512),(2,1024),(2,2048)]
Ls = [2,3,4,5,6,7,8,9]
wds = [1e-3,1e-4,1e-5]
epochs = 60100

### data from linear and relu activations

In [3]:
job_name = "test_GPU"
testMSEs = {}
trainMSEs = {}
weightdecays = {}
learningrates = {}
files_found_list = []
for r,n in rnvals:
    for L in Ls:
        for wd in wds:
            try:
                paramname = job_name+f"/N{n}_L{L}_r{r}_wd{wd}_epochs{epochs}"#f"r{r}_N{n}"
                testMSEs[r,n,L,wd] = np.load(paramname+"testMSE.npy",allow_pickle=True).item()
                trainMSEs[r,n,L,wd] = np.load(paramname+"trainMSEs.npy",allow_pickle=True)
                weightdecays[r,n,L,wd] = np.load(paramname+"weightdecays.npy",allow_pickle=True)
                learningrates[r,n,L,wd] = np.load(paramname+"learningrates.npy",allow_pickle=True)
            except FileNotFoundError as e:
                print(e)
            else:
                files_found_list.append((r,n,L,wd))

[Errno 2] No such file or directory: 'test_GPU/N64_L2_r1_wd0.0001_epochs60100testMSE.npy'
[Errno 2] No such file or directory: 'test_GPU/N64_L2_r1_wd1e-05_epochs60100testMSE.npy'
[Errno 2] No such file or directory: 'test_GPU/N64_L3_r1_wd0.001_epochs60100testMSE.npy'
[Errno 2] No such file or directory: 'test_GPU/N64_L3_r1_wd0.0001_epochs60100testMSE.npy'
[Errno 2] No such file or directory: 'test_GPU/N64_L3_r1_wd1e-05_epochs60100testMSE.npy'
[Errno 2] No such file or directory: 'test_GPU/N64_L4_r1_wd0.001_epochs60100testMSE.npy'
[Errno 2] No such file or directory: 'test_GPU/N64_L4_r1_wd0.0001_epochs60100testMSE.npy'
[Errno 2] No such file or directory: 'test_GPU/N64_L4_r1_wd1e-05_epochs60100testMSE.npy'
[Errno 2] No such file or directory: 'test_GPU/N64_L5_r1_wd0.001_epochs60100testMSE.npy'
[Errno 2] No such file or directory: 'test_GPU/N64_L5_r1_wd0.0001_epochs60100testMSE.npy'
[Errno 2] No such file or directory: 'test_GPU/N64_L5_r1_wd1e-05_epochs60100testMSE.npy'
[Errno 2] No such

In [4]:
files_found_list

[(1, 64, 2, 0.001), (2, 64, 2, 0.001)]

In [5]:
def Llayers(L,d=20,width=1000):
    #construct L-1 linear layers; bias term only on last linear layer
    if L < 2:
        raise ValueError("L must be at least 2")
    if L == 2:
        linear_layers = [nn.Linear(d,width,bias=True)]
    if L > 2:
        linear_layers = [nn.Linear(d,width,bias=False)]
        for l in range(L-3):
            linear_layers.append(nn.Linear(width,width,bias=False))
        linear_layers.append(nn.Linear(width,width,bias=True))

    relu = nn.ReLU()

    last_layer = nn.Linear(width,1)

    layers = linear_layers + [relu,last_layer]

    return nn.Sequential(*layers)

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [7]:
ls N64_L2_r1_wd0.001_epochs60100*

ls: cannot access 'N64_L2_r1_wd0.001_epochs60100*': No such file or directory


In [8]:
files_found_list = []
models = {}
for r,n in rnvals:
    for L in Ls:
        for wd in wds:
            try:
                print(r,n,L,wd)
                paramname = job_name+f"/N{n}_L{L}_r{r}_wd{wd}_epochs60100"#f"r{r}_N{n}"
                models[r,n,L,wd] = Llayers(L,width=1000)
                models[r,n,L,wd].to(device)
                models[r,n,L,wd].load_state_dict(torch.load(paramname+"model.pt"))
                models[r,n,L,wd].eval()
            except FileNotFoundError as e:
                print(e)
            except RuntimeError as e:
                print(e)
            else:
                files_found_list.append((r,n,L,wd))

1 64 2 0.001
1 64 2 0.0001
[Errno 2] No such file or directory: 'test_GPU/N64_L2_r1_wd0.0001_epochs60100model.pt'
1 64 2 1e-05
[Errno 2] No such file or directory: 'test_GPU/N64_L2_r1_wd1e-05_epochs60100model.pt'
1 64 3 0.001
[Errno 2] No such file or directory: 'test_GPU/N64_L3_r1_wd0.001_epochs60100model.pt'
1 64 3 0.0001
[Errno 2] No such file or directory: 'test_GPU/N64_L3_r1_wd0.0001_epochs60100model.pt'
1 64 3 1e-05
[Errno 2] No such file or directory: 'test_GPU/N64_L3_r1_wd1e-05_epochs60100model.pt'
1 64 4 0.001
[Errno 2] No such file or directory: 'test_GPU/N64_L4_r1_wd0.001_epochs60100model.pt'
1 64 4 0.0001
[Errno 2] No such file or directory: 'test_GPU/N64_L4_r1_wd0.0001_epochs60100model.pt'
1 64 4 1e-05
[Errno 2] No such file or directory: 'test_GPU/N64_L4_r1_wd1e-05_epochs60100model.pt'
1 64 5 0.001
[Errno 2] No such file or directory: 'test_GPU/N64_L5_r1_wd0.001_epochs60100model.pt'
1 64 5 0.0001
[Errno 2] No such file or directory: 'test_GPU/N64_L5_r1_wd0.0001_epochs6010

In [9]:
files_found_list

[(1, 64, 2, 0.001), (2, 64, 2, 0.001)]

## create pandas table

In [10]:
res = {
  "r"                 : [r                         for r,n,L,wd in files_found_list],# + [r                         for wd in wds for r,n in relu_rnvals],
  "n"                 : [n                         for r,n,L,wd in files_found_list],# + [n                         for wd in wds for r,n in relu_rnvals],
  "L"                 : [L                         for r,n,L,wd in files_found_list],# + [4                         for wd in wds for r,n in relu_rnvals],
  "lambda"            : [wd                        for r,n,L,wd in files_found_list],# + [wd                        for wd in wds for r,n in relu_rnvals],
  "Learning Rate"     : [learningrates[r,n,L,wd]   for r,n,L,wd in files_found_list],# + [RELUlearningrates[r,n][4][wd] for wd in wds for r,n in relu_rnvals],
  "Train MSE"         : [trainMSEs[r,n,L,wd]       for r,n,L,wd in files_found_list],# + [RELUtrainMSEs[r,n][4][wd]     for wd in wds for r,n in relu_rnvals],
  "Weight Decay"      : [weightdecays[r,n,L,wd]    for r,n,L,wd in files_found_list],# + [RELUweightdecays[r,n][4][wd]  for wd in wds for r,n in relu_rnvals],
  "Model"             : [models[r,n,L,wd]          for r,n,L,wd in files_found_list],# + [RELUmodels[r,n,4,wd]          for wd in wds for r,n in relu_rnvals],
  "Test MSE"          : [testMSEs[r,n,L,wd]        for r,n,L,wd in files_found_list],# + [RELUtestMSEs[r,n][4][wd].item()      for wd in wds for r,n in relu_rnvals],
  "Activations"       : ["linear and relu"         for r,n,L,wd in files_found_list],# + ["relu only"         for wd in wds for r,n in relu_rnvals]
}
res = pd.DataFrame(res)
res["Final Train MSE"] = [r[-1] for r in res["Train MSE"]]
res["Final Weight Decay"] = [r[-1] for r in res["Weight Decay"]]
res

,r,n,L,lambda,Learning Rate,Train MSE,Weight Decay,Model,Test MSE,Activations,Final Train MSE,Final Weight Decay
0,1,64,2,0.001,"[1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-...","[46.94083, 46.85972, 46.77882, 46.698128, 46.6...","[1340.811, 1340.6844, 1340.561, 1340.4401, 134...","[Linear(in_features=20, out_features=1000, bia...",0.120361,linear and relu,3.855108e-06,228.707474
1,2,64,2,0.001,"[1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-...","[86.339066, 86.16455, 85.990234, 85.81613, 85....","[1340.9512, 1340.9651, 1340.9836, 1341.0074, 1...","[Linear(in_features=20, out_features=1000, bia...",10.208943,linear and relu,5.894688e-07,322.884521


# filter out bad training losses

In [11]:
res[res["Final Train MSE"] >= 1e-2]

,r,n,L,lambda,Learning Rate,Train MSE,Weight Decay,Model,Test MSE,Activations,Final Train MSE,Final Weight Decay


In [12]:
res = res[res["Final Train MSE"] < 1e-2]
res

,r,n,L,lambda,Learning Rate,Train MSE,Weight Decay,Model,Test MSE,Activations,Final Train MSE,Final Weight Decay
0,1,64,2,0.001,"[1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-...","[46.94083, 46.85972, 46.77882, 46.698128, 46.6...","[1340.811, 1340.6844, 1340.561, 1340.4401, 134...","[Linear(in_features=20, out_features=1000, bia...",0.120361,linear and relu,3.855108e-06,228.707474
1,2,64,2,0.001,"[1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-...","[86.339066, 86.16455, 85.990234, 85.81613, 85....","[1340.9512, 1340.9651, 1340.9836, 1341.0074, 1...","[Linear(in_features=20, out_features=1000, bia...",10.208943,linear and relu,5.894688e-07,322.884521


#  determine the lambda parameter that gets the best test MSE for each (r,n,L)

In [13]:
testmse_vs_lambda = res.pivot_table(values="Test MSE",index = ("r","n","L","Activations"),columns=["lambda"])
testmse_vs_lambda

,,,lambda,0.001
r,n,L,Activations,
1,64,2,linear and relu,0.120361
2,64,2,linear and relu,10.208943


In [14]:
bestlambda = testmse_vs_lambda.idxmin(axis=1)
bestlambda

r  n   L  Activations    
1  64  2  linear and relu    0.001
2  64  2  linear and relu    0.001
dtype: float64

In [15]:
mask = [row["lambda"] == bestlambda[row["r"]][row["n"]][row["L"]][row["Activations"]] for rowindex,row in res.iterrows()]
res = res[mask]
res

,r,n,L,lambda,Learning Rate,Train MSE,Weight Decay,Model,Test MSE,Activations,Final Train MSE,Final Weight Decay
0,1,64,2,0.001,"[1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-...","[46.94083, 46.85972, 46.77882, 46.698128, 46.6...","[1340.811, 1340.6844, 1340.561, 1340.4401, 134...","[Linear(in_features=20, out_features=1000, bia...",0.120361,linear and relu,3.855108e-06,228.707474
1,2,64,2,0.001,"[1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-...","[86.339066, 86.16455, 85.990234, 85.81613, 85....","[1340.9512, 1340.9651, 1340.9836, 1341.0074, 1...","[Linear(in_features=20, out_features=1000, bia...",10.208943,linear and relu,5.894688e-07,322.884521


# Generalization MSE

## generate data

In [16]:
def gen_data(datasetsize,r,seed,trainsize=2**18,testsize=2**10,d=20,funcseed=42,verbose=False,ood=False):
    ##Generate data with a true central subspaces of varying dimensions
    #generate X values for training and test sets
    np.random.seed(seed) #set seed for data generation
    trainX = np.random.rand(d,trainsize).astype(np.float32)[:,:datasetsize] - 0.5 #distributed as U[-1/2, 1/2]
    testX = np.random.rand(d,testsize).astype(np.float32) - 0.5 #distributed as U[-1/2, 1/2]
    #out of distribution datagen
    if ood:
        trainX *= 2 #now distributed as U[-1, 1]
        testX *= 2 #now distributed as U[-1, 1]
    ##for each $r$ value create and store data-gen functions and $y$ evaluations
    #geneate params for functions
    k = d+1
    U = np.load(job_name+f"/r{r}U.npy")
    Sigma = np.load(job_name+f"/r{r}Sigma.npy")
    V = np.load(job_name+f"/r{r}V.npy")
    A = np.load(job_name+f"/r{r}A.npy")
    B = np.load(job_name+f"/r{r}B.npy")
    #crealoadte functions
    def g(z): #active subspace function
        hidden_layer = (U*Sigma)@z
        hidden_layer = hidden_layer.T + B
        hidden_layer = np.maximum(0,hidden_layer).T
        return A@hidden_layer
    def f(x): #teacher network
        z = V.T@x
        return g(z)
    #generate data
    trainY = f(trainX).astype(np.float32)
    testY = f(testX).astype(np.float32)
    #move data to device
    if verbose:
        print("device: {}".format(device))
    trainX = torch.from_numpy(trainX).T.to(device)
    trainY = torch.from_numpy(trainY).to(device)
    testX = torch.from_numpy(testX).T.to(device)
    testY = torch.from_numpy(testY).to(device)
    if verbose:
        print("trainX shape = {} trainY shape = {}".format(
            trainX.shape,
            trainY.shape
        ))
    return trainX,trainY,testX,testY


In [17]:
generalizationX,generalizationYr1 = gen_data(2048,r=1,seed=101)[:2]
generalizationX,generalizationYr2 = gen_data(2048,r=2,seed=101)[:2]
generalizationY = {1:generalizationYr1,2:generalizationYr2}

In [18]:
generalizationX.min(),generalizationX.max()

(tensor(-0.4999, device='cuda:0'), tensor(0.5000, device='cuda:0'))

## compute MSE

In [19]:
with torch.no_grad():
    generalization = []
    for rownum, row in res.iterrows():
        predY = row["Model"](generalizationX)
        err = nn.functional.mse_loss(predY[:,0],generalizationY[row["r"]]).item()
        generalization.append(err)
    res["Generalization MSE"] = generalization

In [20]:
res

,r,n,L,lambda,Learning Rate,Train MSE,Weight Decay,Model,Test MSE,Activations,Final Train MSE,Final Weight Decay,Generalization MSE
0,1,64,2,0.001,"[1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-...","[46.94083, 46.85972, 46.77882, 46.698128, 46.6...","[1340.811, 1340.6844, 1340.561, 1340.4401, 134...","[Linear(in_features=20, out_features=1000, bia...",0.120361,linear and relu,3.855108e-06,228.707474,0.114529
1,2,64,2,0.001,"[1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-...","[86.339066, 86.16455, 85.990234, 85.81613, 85....","[1340.9512, 1340.9651, 1340.9836, 1341.0074, 1...","[Linear(in_features=20, out_features=1000, bia...",10.208943,linear and relu,5.894688e-07,322.884521,10.132702


# Out of Distribution MSE

## generate data

In [21]:
oodX,oodYr1 = gen_data(2048,r=1,seed=101,ood=True)[:2] #generate out of distribution data
oodX,oodYr2 = gen_data(2048,r=2,seed=101,ood=True)[:2]
oodY = {1:oodYr1,2:oodYr2}

In [22]:
oodX.min(),oodX.max()

(tensor(-0.9999, device='cuda:0'), tensor(1.0000, device='cuda:0'))

## compute MSE

In [23]:
with torch.no_grad():
    ood_errs = []
    for rownum, row in res.iterrows():
        predY = row["Model"](oodX)
        err = nn.functional.mse_loss(predY[:,0],oodY[row["r"]]).item()
        ood_errs.append(err)
    res["Out of Distribution MSE"] = ood_errs

In [24]:
res

,r,n,L,lambda,Learning Rate,Train MSE,Weight Decay,Model,Test MSE,Activations,Final Train MSE,Final Weight Decay,Generalization MSE,Out of Distribution MSE
0,1,64,2,0.001,"[1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-...","[46.94083, 46.85972, 46.77882, 46.698128, 46.6...","[1340.811, 1340.6844, 1340.561, 1340.4401, 134...","[Linear(in_features=20, out_features=1000, bia...",0.120361,linear and relu,3.855108e-06,228.707474,0.114529,1.046242
1,2,64,2,0.001,"[1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-...","[86.339066, 86.16455, 85.990234, 85.81613, 85....","[1340.9512, 1340.9651, 1340.9836, 1341.0074, 1...","[Linear(in_features=20, out_features=1000, bia...",10.208943,linear and relu,5.894688e-07,322.884521,10.132702,42.596649


# Active Subspace

## evaluate gradients and compute singular values and active subspaces

In [25]:
generalizationY

{1: tensor([  2.8498,   1.3553,  -7.2570,  ...,   0.7691, -12.6291,   1.3051],
        device='cuda:0'),
 2: tensor([ 1.5574,  7.7949, 19.1428,  ...,  8.1604, 10.2077,  3.3550],
        device='cuda:0')}

In [26]:
grads = []
sv = []
active_subspace = []
subspace_dist = []

for rownum, row in res.iterrows():
    #compute ground truth active subspace
    funcseed = 42
    d = 20
    k = d+1
    U = np.load(f"r{row['r']}U.npy")
    Sigma = np.load(f"r{row['r']}Sigma.npy")
    V = np.load(f"r{row['r']}V.npy")
    W = np.load(f"r{row['r']}W.npy")
    A = np.load(f"r{row['r']}A.npy")
    B = np.load(f"r{row['r']}B.npy")

    #evaluate gradients
    generalizationX.requires_grad = True
    predY = row["Model"](generalizationX)
    grad = torch.autograd.grad(predY, generalizationX,
                            grad_outputs=torch.ones_like(predY),
                            create_graph=True)[0].detach().cpu().numpy()
    grads.append(grad)
    #compute active subspace and singular values
    Uhat,Shat,VhatT = np.linalg.svd(grad)
    Vhat = VhatT.T[:,:row["r"]] #form the basis for the active subspace
    active_subspace.append(Vhat)
    sv.append(Shat)

    subspace_dist.append(np.linalg.norm(V@V.T - Vhat@Vhat.T,2))

res["Gradient Evaluations"] = grads
res["Gradient Singular Values"] = sv
res["Active Subspace"] = active_subspace
res["Active Subspace Distance"] = subspace_dist
res["Active Subspace Alignment Angle (Degrees)"] = np.degrees(np.arcsin(res["Active Subspace Distance"]))

res

,r,n,L,lambda,Learning Rate,Train MSE,Weight Decay,Model,Test MSE,Activations,Final Train MSE,Final Weight Decay,Generalization MSE,Out of Distribution MSE,Gradient Evaluations,Gradient Singular Values,Active Subspace,Active Subspace Distance,Active Subspace Alignment Angle (Degrees)
0,1,64,2,0.001,"[1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-...","[46.94083, 46.85972, 46.77882, 46.698128, 46.6...","[1340.811, 1340.6844, 1340.561, 1340.4401, 134...","[Linear(in_features=20, out_features=1000, bia...",0.120361,linear and relu,3.855108e-06,228.707474,0.114529,1.046242,"[[0.10197979, 0.112861365, -5.711627, -1.60037...","[891.79254, 18.982658, 13.441324, 11.738973, 7...","[[-0.003839484], [0.020108897], [-0.5897444], ...",0.036544,2.094260
1,2,64,2,0.001,"[1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-04, 1e-...","[86.339066, 86.16455, 85.990234, 85.81613, 85....","[1340.9512, 1340.9651, 1340.9836, 1341.0074, 1...","[Linear(in_features=20, out_features=1000, bia...",10.208943,linear and relu,5.894688e-07,322.884521,10.132702,42.596649,"[[2.2919116, -0.02091679, 1.0606848, -2.109074...","[397.38116, 220.02351, 115.69383, 81.82117, 66...","[[-0.24357323, 0.4447303], [0.048261184, -0.18...",0.712961,45.476307


## plot of singular values

In [27]:
# colors = {(1,64):"tab:blue",(1,2048):"tab:cyan",(2,64):"tab:orange",(2,128):"tab:pink",(2,256):"tab:red",(2,2048):"tab:purple"}

In [28]:
res.n.unique()

array([64])

In [29]:
f, ax = plt.subplots(nrows=len(res.r.unique()), ncols=len(res.n.unique()), sharex=True, sharey=True, figsize=(10,4.8))#len(res.r.unique()),len(res.n.unique())
for rownum,row in res.iterrows():
    whichrow = np.where(row['r'] == res.r.unique())[0][0]
    whichcol = np.where(row['n'] == res.n.unique())[0][0]
    print(whichrow,whichcol)
    #whichax = row['L'] - 2
#     color = colors[(row['L'])]
    print(rf"{row['r']},{row['n']},{row['L']}",row["Gradient Singular Values"]/np.sqrt(2048),whichrow,whichcol)
    ax[whichrow,whichcol].semilogy(row["Gradient Singular Values"]/np.sqrt(2048),label=rf"$L={row['L']}$",linewidth=1,alpha=0.7,marker=".")
    ax[whichrow,whichcol].set_xticks(list(range(3,20,4)),list(range(4,21,4)))
    ax[whichrow,whichcol].set_ylim(10**-9,10**3)
    ax[0,whichcol].set_title(rf"$n={row['n']}$")
    ax[-1,whichcol].set_xlabel(rf"Index, $k$")
# plt.subplot(2,len(res.n.unique()),1)
# leg = plt.legend()
# leg = plt.legend(bbox_to_anchor=(-0.5, 1.3))
# leg.get_frame().set_edgecolor('b')
# leg.get_frame().set_linewidth(0.0)
# plt.subplot(2,len(res.n.unique()),1)
# plt.ylabel(r"$r=1$"+"\n"+r"$\sigma_k(\hat f;\rho)$")
# plt.yticks([10**p for p in range(-8,3,2)])
# plt.subplot(2,len(res.n.unique()),len(res.n.unique())+1)
# plt.ylabel(r"$r=2$"+"\n"+r"$\sigma_k(\hat f;\rho)$")
# plt.yticks([10**p for p in range(-8,3,2)])
# plt.suptitle("Singular Values of Trained Networks")
# plt.tight_layout()
plt.savefig(job_name+"/sv.png",dpi=300,bbox_extra_artists=(leg,), bbox_inches='tight')
plt.show()

SyntaxError: '(' was never closed (3561929491.py, line 1)

#  determine the L parameter that gets the best test MSE for each (r,n)

In [ ]:
testmse_vs_L = res.pivot_table(values="Test MSE",index = ("r","n","Activations"),columns=["L"])
testmse_vs_L = testmse_vs_L.iloc[:,1:]
testmse_vs_L

In [ ]:
bestL = testmse_vs_L.idxmin(axis=1)
bestL

In [ ]:
mask = [row["L"] == bestL[row["r"]][row["n"]][row["Activations"]] for rowindex,row in res.iterrows()]
bestLres = res[mask]
bestLres

# Plots of L vs Test error and n vs Generalization metrics with/without linear layers

In [ ]:
res.r.unique()

In [ ]:
f, ax = plt.subplots(ncols=len(res.r.unique()),nrows=1, sharex=True, sharey=True, figsize=(10,4.8))
for rnum,r in enumerate(res.r.unique()):
    for n in res.n.unique():
        res_rn = res[(res.r == r) * (res.n == n)]
        ax[rnum].scatter(res_rn.L,res_rn[["Test MSE"]])
        ax[rnum].semilogy(res_rn.L,res_rn[["Test MSE"]],label=rf"$n={n}$")
    ax[rnum].set_xlabel("$L$ number of layers")
    ax[rnum].set_title(rf"$r={r}$")
ax[0].legend()
ax[0].set_ylabel("Validation MSE")
f.suptitle(rf"Validation MSE for best $\lambda$ values")
f.tight_layout()
f.savefig(job_name+f"/Validation MSE.png",dpi=300)
plt.show()

In [ ]:
for metric in ['Generalization MSE', 'Out of Distribution MSE','Active Subspace Alignment Angle (Degrees)']:
    f, ax = plt.subplots(ncols=len(res.r.unique()),nrows=1, sharex=True, sharey=True, figsize=(10,4.8))
    for rnum,r in enumerate(res.r.unique()):
        plt.subplot(1,2,r)
        #best L
        curr = bestLres[bestLres.r == r]
        ax[rnum].scatter(curr.n,curr[[metric]])
        ax[rnum].plot(curr.n,curr[[metric]],label=rf"linear layers")
        #no linear layers
        no_lin_layers = res[(res.r == r) * (res.L == 2)]
        ax[rnum].scatter(no_lin_layers.n,no_lin_layers[[metric]])
        ax[rnum].plot(no_lin_layers.n,no_lin_layers[[metric]],label=rf"no linear layers")
        #plotting extras
        ax[rnum].set_xlabel("$n$ number of samples")
        ax[rnum].set_xscale("log",base=2)
        if metric != 'Active Subspace Alignment Angle (Degrees)':
            ax[rnum].set_yscale("log",base=10)
        ax[rnum].set_title(rf"$r={r}$")
    ax[0].set_ylabel(f"{metric}")
    ax[0].legend()
    plt.suptitle(f"{metric}" + r" with best $(\lambda,L)$ tuning")
    plt.tight_layout()
    plt.savefig(job_name+f"/{metric}.png",dpi=300)
    plt.show()

# Final Table

In [ ]:
res.loc[(res.L == 2),"Activations"] = "Shallow ReLU"

In [ ]:
bestLres = pd.concat((res[(res.L == 2)],bestLres))

In [ ]:
bestLres.shape

In [ ]:
pivot_table = bestLres.pivot_table(index=("r","n","Activations"))
print(pivot_table.shape)
pivot_table

In [ ]:
pivot_table.idxmin(axis=0)

In [ ]:
pivot_table_latex = pivot_table.drop(["Final Weight Decay"],axis=1)
pivot_table_latex = pivot_table_latex[["L","lambda","Final Train MSE","Generalization MSE","Out of Distribution MSE","Active Subspace Alignment Angle (Degrees)"]]
pivot_table_latex.to_latex(float_format="%.2e")

In [ ]:
pivot_table_latex

# Training Time Plots

## Train MSE v Epoch

In [ ]:
f, ax = plt.subplots(nrows=len(res.r.unique()), ncols=len(res.n.unique()), sharex=True, sharey=True, figsize=(20,10))#len(res.r.unique()),len(res.n.unique())
plt.figure(figsize=(10,10))
for rownum,row in res.iterrows():
    if row["L"] <= 4:
        whichrow = np.where(row['r'] == res.r.unique())[0][0]
        whichcol = np.where(row['n'] == res.n.unique())[0][0]
        ax[whichrow,whichcol].semilogy(row["Train MSE"],label=rf"$L = {row['L']}$",linewidth=1,alpha=0.5)
        ax[whichrow,whichcol].set_title(rf"$r = {row['r']},n = {row['n']}$")
        ax[whichrow,whichcol].set_xlabel("Epoch")
ax[0,0].legend()
f.suptitle("Train MSE v Epoch")
f.savefig(job_name+"/trainmse234.png",dpi=300)
plt.show()

In [ ]:
f, ax = plt.subplots(nrows=len(res.r.unique()), ncols=len(res.n.unique()), sharex=True, sharey=True, figsize=(20,10))#len(res.r.unique()),len(res.n.unique())
plt.figure(figsize=(10,10))
for rownum,row in res.iterrows():
    if 4 < row["L"] <= 7:
        whichrow = np.where(row['r'] == res.r.unique())[0][0]
        whichcol = np.where(row['n'] == res.n.unique())[0][0]
        ax[whichrow,whichcol].semilogy(row["Train MSE"],label=rf"$L = {row['L']}$",linewidth=1,alpha=0.5)
        ax[whichrow,whichcol].set_title(rf"$r = {row['r']},n = {row['n']}$")
        ax[whichrow,whichcol].set_xlabel("Epoch")
ax[0,0].legend()
f.suptitle("Train MSE v Epoch")
f.savefig(job_name+"/trainmse567.png",dpi=300)
plt.show()

In [ ]:
f, ax = plt.subplots(nrows=len(res.r.unique()), ncols=len(res.n.unique()), sharex=True, sharey=True, figsize=(20,10))
plt.figure(figsize=(10,10))
for rownum,row in res.iterrows():
    if row["L"] > 7:
        whichrow = np.where(row['r'] == res.r.unique())[0][0]
        whichcol = np.where(row['n'] == res.n.unique())[0][0]
        ax[whichrow,whichcol].semilogy(row["Train MSE"],label=rf"$L = {row['L']}$",linewidth=1,alpha=0.5)
        ax[whichrow,whichcol].set_title(rf"$r = {row['r']},n = {row['n']}$")
        ax[whichrow,whichcol].set_xlabel("Epoch")
ax[0,0].legend()
f.suptitle("Train MSE v Epoch")
f.savefig(job_name+"/trainmse8910.png",dpi=300)
plt.show()

## Weight Decay v Epoch

In [ ]:
f, ax = plt.subplots(len(res.r.unique()), ncols=len(res.n.unique()), sharex=True, sharey=True, figsize=(20,10))#len(res.r.unique()),len(res.n.unique())
plt.figure(figsize=(10,10))
for rownum,row in res.iterrows():
    whichrow = np.where(row['r'] == res.r.unique())[0][0]
    whichcol = np.where(row['n'] == res.n.unique())[0][0]
    ax[whichrow,whichcol].semilogy(row["Weight Decay"],label=rf"$L = {row['L']}$",linewidth=1,alpha=0.7)
    ax[whichrow,whichcol].set_title(rf"$r = {row['r']},n = {row['n']}$")
    ax[whichrow,whichcol].set_xlabel("Epoch")
ax[0,0].legend()
f.suptitle("Weight Decay v Epoch")
f.savefig(job_name+"/weightdecay.png",dpi=300)
plt.show()

## learning rates

In [ ]:
f, ax = plt.subplots(nrows=len(res.r.unique()), ncols=len(res.n.unique()), sharex=True, sharey=True, figsize=(20,10))#len(res.r.unique()),len(res.n.unique())
plt.figure(figsize=(10,10))
for rownum,row in res.iterrows():
    whichrow = np.where(row['r'] == res.r.unique())[0][0]
    whichcol = np.where(row['n'] == res.n.unique())[0][0]
    ax[whichrow,whichcol].semilogy(row["Learning Rate"],label=rf"$L = {row['L']}$",linewidth=1,alpha=0.7)
    ax[whichrow,whichcol].set_title(rf"$r = {row['r']},n = {row['n']}$")
    ax[whichrow,whichcol].set_xlabel("Epoch")
ax[0,0].legend()
f.suptitle("Learning Rate v Epoch")
f.savefig(job_name+"/LearningRate.png",dpi=300)
plt.show()